## Summarizing a text using an LLM

As an LLM "understands" a language, it can be suited for tasks like translation or summarization.

In this Notebook, we are going to use our LLM to summarize some texts, especially claims examples.

### Requirements and Imports

If you have selected the right workbench image to launch as per the Lab's instructions, you should already have all the needed libraries. If not uncomment the first line in the next cell to install all the right packages.

In [1]:
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt # Uncomment only if you have not selected the right workbench image

import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
import requests
import json
from requests.exceptions import ConnectionError, Timeout, RequestException

### Langchain pipeline

Again, we are going to use Langchain to define our summarization pipeline.

In [2]:
# LLM Inference Server URL
inference_server_url = "http://llm.ic-shared-llm.svc.cluster.local:3000/"

# LLM definition
llm = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url,
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

The **template** we will use is now formatted for this specific summarization task.

In [3]:
template="""<s>[INST]
You are a helpful, respectful and honest assistant.
Always assist with care, respect, and truth. Respond with utmost utility yet securely.
Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.
I will give you a text that you must summarize as best as you can.

### TEXT:
{input}

### SUMMARY:
[/INST]
"""
PROMPT = PromptTemplate(input_variables=["input"], template=template)

And we can now create the **conversation** object that we will use to query the model.

In [4]:
conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False
                        )

We are now ready to query the model!

In the `claims` folder we have JSON files with examples of claims that could be received. We are going to read those files, display them, then the summary that the LLM made.

In [5]:
# Define the base URL of the service
base_url = 'http://backend:8080/claim'


In [6]:
# List of URLs to fetch the JSON files from
urls = [
    'http://backend:8080/claim/claim1.json',
    'http://backend:8080/claim/claim2.json',
    'http://backend:8080/claim/claim3.json'
]

claims = {}

for url in urls:
    # Extract the filename from the URL
    filename = os.path.basename(url)
    
    # Make an HTTP GET request to fetch the JSON file
    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses
    
    # Load the JSON data from the response
    data = response.json()
    
    # Store the data in the claims dictionary
    claims[filename] = data

# Now the claims dictionary is populated with the data from the URLs

In [7]:
# Read the claims and populate a dictionary
claims_path = 'claims'
onlyfiles = [f for f in listdir(claims_path) if isfile(join(claims_path, f))]

claims = {}

for filename in onlyfiles:
    # Opening JSON file
    with open(os.path.join(claims_path, filename), 'r') as file:
        data = json.load(file)
    claims[filename] = data


In [8]:
for filename in onlyfiles:
    print(f"***************************")
    print(f"* Claim: {filename}")
    print(f"***************************")
    print("Original content:")
    print("-----------------")
    print(f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}\n\n")
    print('Summary:')
    print("--------")
    summary_input = f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}"
    conversation.predict(input=summary_input);
    print("\n\n                          ----====----\n")

***************************
* Claim: claim2.json
***************************
Original content:
-----------------
Subject: Urgent: Unacceptable Delay and Lack of Communication Regarding Claim #XYZ789
Content:
Dear Prestige Insurances,

I am writing to express my extreme dissatisfaction with the appalling service I have received concerning my recent claim, reference number #XYZ789. The lack of communication and delayed response from your company is utterly unacceptable, and I demand immediate attention to rectify this matter.

The accident occurred on January 15, 2024, at approximately 3:45 PM, near the intersection of Maple Street and Oak Avenue in Rivertown. My vehicle, a Silver Hawk GT, was struck by another driver, identified as Samantha Reynolds, who ran a red light at the aforementioned intersection. This incident resulted in substantial damage to the front end of my car, including severe structural damage and airbag deployment.

I reported the claim immediately after the accident,

We will come back to this notebook at section 3.5 for some exercises, so you can leave it open for the moment.